In [0]:
# service principal for integrating with ADLS and access it's data

spark.conf.set("fs.azure.account.auth.type.hpadlsacc.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hpadlsacc.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "app-key"))
spark.conf.set("fs.azure.account.oauth2.client.secret.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "service-cred"))
tenant_id = dbutils.secrets.get("hc-secret-scope", "dir-id")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hpadlsacc.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Creating claims table in Silver layer

In [0]:
# defining the source path for claims data
src_path = "abfss://bronze@hpadlsacc.dfs.core.windows.net/claims/"

# reading the path
claims_df = spark.read.format("parquet").load(src_path)
display(claims_df)

# to do sql operations, we need to register the dataframe as a temp view
claims_df.createOrReplaceTempView("claims")

In [0]:
%sql
-- creating the temp view for quality check
-- adding a new column 'is_quarantined' to check for null values
-- if any primary/important column null then the particular record should be quarantined(True)
-- else Not(False)

CREATE OR REPLACE TEMP VIEW quality_checks AS
SELECT 
 CONCAT(ClaimID,'-', datasource) AS ClaimID,
 ClaimID AS SRC_ClaimID,
 TransactionID,
 PatientID,
 EncounterID,
 ProviderID,
 DeptID,
 cast(ServiceDate as date) ServiceDate,
 cast(ClaimDate as date) ClaimDate,
 PayorID,
 ClaimAmount,
 PaidAmount,
 ClaimStatus,
 PayorType,
 Deductible,
 Coinsurance,
 Copay,
 cast(InsertDate as date) as SRC_InsertDate,
 cast(ModifiedDate as date) as SRC_ModifiedDate,
 datasource,
 CASE 
     WHEN ClaimID IS NULL OR TransactionID IS NULL OR PatientID IS NULL or ServiceDate IS NULL THEN TRUE
     ELSE FALSE
 END AS is_quarantined
FROM claims;


In [0]:
%sql
-- displaying the quality_checks table records
select * from quality_checks;

In [0]:
%sql
-- creating the silver.claims external table 

CREATE TABLE IF NOT EXISTS silver.claims (
  ClaimID string,
  SRC_ClaimID string,
  TransactionID string,
  PatientID string,
  EncounterID string,
  ProviderID string,
  DeptID string,
  ServiceDate date,
  ClaimDate date,
  PayorID string,
  ClaimAmount string,
  PaidAmount string,
  ClaimStatus string,
  PayorType string,
  Deductible string,
  Coinsurance string,
  Copay string,
  SRC_InsertDate date,
  SRC_ModifiedDate date,
  datasource string,
  is_quarantined boolean,
  audit_insertdate timestamp,
  audit_modifieddate timestamp,
  is_current boolean
)
USING DELTA
LOCATION "abfss://silver@hpadlsacc.dfs.core.windows.net/claims";


In [0]:
%sql
-- Step 1: based on condition ClaimID should be simliar and is_current should be true (i.e currently that specific record is active)
-- Mark that existing records as historical (is_current = false) for patients that will be updated
-- target.is_current = false,
-- target.modified_date = current_timestamp()

MERGE INTO silver.claims AS target
USING quality_checks AS source
ON target.ClaimID = source.ClaimID AND target.is_current = true
WHEN MATCHED AND (
    target.SRC_ClaimID != source.SRC_ClaimID OR
    target.TransactionID != source.TransactionID OR
    target.PatientID != source.PatientID OR
    target.EncounterID != source.EncounterID OR
    target.ProviderID != source.ProviderID OR
    target.DeptID != source.DeptID OR
    target.ServiceDate != source.ServiceDate OR
    target.ClaimDate != source.ClaimDate OR
    target.PayorID != source.PayorID OR
    target.ClaimAmount != source.ClaimAmount OR
    target.PaidAmount != source.PaidAmount OR
    target.ClaimStatus != source.ClaimStatus OR
    target.PayorType != source.PayorType OR
    target.Deductible != source.Deductible OR
    target.Coinsurance != source.Coinsurance OR
    target.Copay != source.Copay OR
    target.SRC_InsertDate != source.SRC_InsertDate OR
    target.SRC_ModifiedDate != source.SRC_ModifiedDate OR
    target.datasource != source.datasource OR
    target.is_quarantined != source.is_quarantined
) THEN
  UPDATE SET
    target.is_current = false,
    target.audit_modifieddate = current_timestamp()


In [0]:
%sql
-- Step 2: Insert new and updated records into the Delta table, marking them as current
-- that is old record is updated with new records
-- based on condition claimID should be simliar and is_current should be true (i.e currently that specific record is active)
-- because the condition will not satisfy
-- inserting the new records which are not present in the silver table and updating the old records

MERGE INTO silver.claims AS target
USING quality_checks AS source
ON target.ClaimID = source.ClaimID AND target.is_current = true
WHEN NOT MATCHED THEN
  INSERT (
    ClaimID,
    SRC_ClaimID,
    TransactionID,
    PatientID,
    EncounterID,
    ProviderID,
    DeptID,
    ServiceDate,
    ClaimDate,
    PayorID,
    ClaimAmount,
    PaidAmount,
    ClaimStatus,
    PayorType,
    Deductible,
    Coinsurance,
    Copay,
    SRC_InsertDate,
    SRC_ModifiedDate,
    datasource,
    is_quarantined,
    audit_insertdate,
    audit_modifieddate,
    is_current
  )
  VALUES (
    source.ClaimID,
    source.SRC_ClaimID,
    source.TransactionID,
    source.PatientID,
    source.EncounterID,
    source.ProviderID,
    source.DeptID,
    source.ServiceDate,
    source.ClaimDate,
    source.PayorID,
    source.ClaimAmount,
    source.PaidAmount,
    source.ClaimStatus,
    source.PayorType,
    source.Deductible,
    source.Coinsurance,
    source.Copay,
    source.SRC_InsertDate,
    source.SRC_ModifiedDate,
    source.datasource,
    source.is_quarantined,
    current_timestamp(),
    current_timestamp(),
    true
  );


In [0]:
%sql
-- pulling some records
select * from  silver.claims

In [0]:
%sql
-- drop table silver.claims